In [1]:
!pip install -r requirements.txt

In [15]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *

In [16]:
bus_hqtc = gpd.read_parquet('./data/bus/shape_hqta_dissolve.parquet')
bus_hqtc = bus_hqtc[bus_hqtc['hq_transit_corr']]
bus_hqtc['hqta_type'] = 'hq_transit_corr'

In [17]:
rail_ferry_brt_stops = gpd.read_parquet('./data/rail_ferry_brt/rail_brt_ferry.parquet')
rail_ferry_brt_stops['hqta_type'] = 'major_transit_stop'

### High Quality Transit Areas Relevant Statutes

[PRC 21155](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21155.&lawCode=PRC)
* _(3) be within one-half mile of a major transit stop or high-quality transit corridor included in a regional transportation plan._
* Major transit stop definition: _A major transit stop is as defined in Section 21064.3, except that, for purposes of this section, it also includes major transit stops that are included in the applicable regional transportation plan_
* High-quality transit corridor definition: _For purposes of this section, a high-quality transit corridor means a corridor with fixed route bus service with service intervals no longer than 15 minutes during peak commute hours._
    * Unable to locate definition of "peak commute hours"

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

## Bus Major Stops

In [18]:
bus_hqtc.head(3)

,hq_transit_corr,shape_id,geometry,level_0,calitp_itp_id,hqta_segment_id,n_trips,segment_sequence,stop_id,am_peak,pm_peak,hqta_type
2464,True,0500,"MULTIPOLYGON (((193308.696 -465893.120, 193308...",0,142,2197551287,134.0,0,3977,9.0,8.0,hq_transit_corr
2466,True,10,"MULTIPOLYGON (((170627.569 -429628.875, 170627...",1,243,2088216280,64.0,2,604,4.7,5.0,hq_transit_corr
2467,True,100706_SEPT21,"MULTIPOLYGON (((149722.349 -435846.031, 149624...",0,182,2368710740,120.0,1,16374,6.0,8.0,hq_transit_corr


In [19]:
octa = bus_hqtc >> filter(_.calitp_itp_id == '142')
octa.head(3)

,hq_transit_corr,shape_id,geometry,level_0,calitp_itp_id,hqta_segment_id,n_trips,segment_sequence,stop_id,am_peak,pm_peak,hqta_type
2464,True,0500,"MULTIPOLYGON (((193308.696 -465893.120, 193308...",0,142,2197551287,134.0,0,3977,9.0,8.0,hq_transit_corr
2840,True,2902,"MULTIPOLYGON (((185843.557 -482535.788, 185898...",27,142,369979426,121.0,28,4309,7.0,6.0,hq_transit_corr
2933,True,4001,"MULTIPOLYGON (((199427.290 -480391.891, 199427...",0,142,3081829928,45.0,2,3532,5.0,5.0,hq_transit_corr


In [23]:
over = gpd.overlay(octa, octa, how='intersection')

In [32]:
x = octa[octa.shape_id == '9430']
y = octa[octa.shape_id == '6422']

In [33]:
df = x.clip(y)
df.geometry = df.geometry.buffer(50)

In [109]:
gdf = bus_hqtc
output = gpd.GeoDataFrame()

def find_intersections(row):
    global output
    # display(row)
    # display(row.shape_id)
    this_row = gdf >> filter(_.shape_id == row.shape_id)
    not_this_row = gdf >> filter(_.shape_id != row.shape_id)
    # print(type(filtered))
    clip_row = gpd.clip(this_row, not_this_row)
    output = output.append(clip_row)
    return

In [110]:
_test = bus_hqtc.apply(find_intersections, axis=1)

In [111]:
output.geometry = output.geometry.buffer(50)

In [116]:
# output.to_parquet('finding_intersections.parquet')

In [117]:
output = gpd.read_parquet('finding_intersections.parquet')

In [114]:
map_hqta(output, 'shape_id')

Map(center=[33.78198752372606, -117.88832768212126], controls=(ZoomControl(options=['position', 'zoom_in_text'…

#### Definitions and Output:

* hqta_type: major_transit_stop
* stop_id: one stop id... (not ideal, but oh well)
* geometry: .buffer(700)?
* _can alway pull more info spatially_

### Combining and Buffering

* General buffer distance: 1/2mi ~= 805 meters
* Bus corridors are already buffered 50 meters, so will buffer 755 meters

In [34]:
bus_hqtc.geometry = bus_hqtc.geometry.buffer(755)
rail_ferry_brt_stops.geometry = rail_ferry_brt_stops.geometry.buffer(805)

In [35]:
all_hqta = bus_hqtc.append(rail_ferry_brt_stops).fillna('')

In [36]:
all_hqta = all_hqta[['calitp_itp_id', 'stop_id', 'geometry', 'hqta_type']]
all_hqta = all_hqta.reset_index(drop=True)
all_hqta['calitp_itp_id'] = all_hqta['calitp_itp_id'].astype('int64')

### Format for export

In [37]:
all_hqta

,calitp_itp_id,stop_id,geometry,hqta_type
0,1,0255,"MULTIPOLYGON (((170345.013 -468746.416, 170344...",hq_transit_corr
1,142,3977,"MULTIPOLYGON (((211656.399 -486108.638, 211648...",hq_transit_corr
2,243,315,"MULTIPOLYGON (((176140.516 -428554.874, 176087...",hq_transit_corr
3,243,604,"POLYGON ((169813.526 -429207.132, 169810.845 -...",hq_transit_corr
4,1,2029,"POLYGON ((160196.861 -459719.052, 160180.987 -...",hq_transit_corr
...,...,...,...,...
2203,2,890002,"POLYGON ((-209416.680 -22028.923, -209420.557 ...",major_transit_stop
2204,2,890001,"POLYGON ((-209486.852 -21944.196, -209490.728 ...",major_transit_stop
2205,2,890001,"POLYGON ((-209486.852 -21944.196, -209490.728 ...",major_transit_stop
2206,2,12030042,"POLYGON ((-200930.070 -22595.234, -200933.947 ...",major_transit_stop


### Mapping

In [7]:
def map_hqta(gdf, mouseover=None):
    global nix_list
    nix_list = []
    
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
    gdf = gdf.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    if gdf.geometry.iloc[0].geom_type == 'Point':
        gdf.geometry = gdf.geometry.buffer(200)
    
    x = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.x
    y = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
            
        def add_to_nix(feature, **kwargs):
            nix_list.append(feature['properties'][mouseover])
            
    if 'hq_transit_corr' in gdf.columns:
        geo_data_hq = GeoData(geo_dataframe = gdf[gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'HQTA')
        #a8ddb5
        geo_data_not_hq = GeoData(geo_dataframe = gdf[~gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#fec44f',
                                            'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'non-HQTA')

        m.add_layer(geo_data_hq)
        m.add_layer(geo_data_not_hq)
    
    else:
    
        geo_data_hq = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'gdf')
        m.add_layer(geo_data_hq)
    
    if mouseover:
        geo_data_hq.on_hover(update_html)
        geo_data_hq.on_hover(add_to_nix)

    m.add_control(LayersControl())

    return m

In [39]:
map_hqta(all_hqta)

Map(center=[33.79897878118749, -118.16219937789772], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [41]:
all_hqta.to_parquet('./data/combined/ca_high_quality_transit.parquet')
all_hqta.to_file('./data/combined/ca_high_quality_transit.geojson', driver='GeoJSON')

/tmp/ipykernel_72/793328695.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  all_hqta.to_parquet('./data/combined/ca_high_quality_transit.parquet')


## Static Map Images (all HQTAs)

![bay](img/bay_valley_all.png)

![fresno](img/fres_all.png)

![san_diego](img/sd_all.png)